In [70]:
import numpy as np
import math

functionCalculated = 0

def f(x):
    x1 = x[0]
    x2 = x[1]
    global functionCalculated
    functionCalculated += 1
    return (x1-1)**2 + (x2-1)**2

In [71]:
# print(f"{h}, {h*np.array([1, 0])}, {h*np.array([0, 1])}")
def fderright(x, fx, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - fx) / h
    gradFx2 = (f(x + h*np.array([0, 1])) - fx) / h
    return np.array([gradFx1, gradFx2])

def fderleft(x, fx, h):
    gradFx1 = (fx - f(x - h*np.array([1, 0]))) / h
    gradFx2 = (fx - f(x - h*np.array([0, 1]))) / h
    return np.array([gradFx1, gradFx2])

def fdercenter(x, h):
    gradFx1 = (f(x + h*np.array([1, 0])) - f(x - h*np.array([1, 0]))) / (2*h)
    gradFx2 = (f(x + h*np.array([0, 1])) - f(x - h*np.array([0, 1]))) / (2*h)
    return np.array([gradFx1, gradFx2])

In [72]:
def norm(x):
    x1 = x[0]
    x2 = x[1]
    return math.sqrt(x1**2 + x2**2)

def stopCriteria1(xk, fxk, xk_1, fxk_1, e):
    if ((norm((xk_1 - xk)) / norm(xk)) <= e) and (math.fabs(fxk_1 - fxk) <= e):
        return True
    return False

def stopCriteria2(gradFxk, e):
    if (norm(gradFxk) <= e):
        return True
    return False

In [73]:
# Code for finding optimal Lambda
functionCalculatedSven = 0
svenCoefLambda = 0.1
l0 = 0

def f(x):
    x1 = x[0]
    x2 = x[1]
    return 3*(x1-15)**2 -x1*x2 +4*x2**2

def FLambda(xk, l, Sk):
    return f(xk + l*Sk)

def sven(xk, Sk):
    deltaL = svenCoefLambda * (norm(xk) / norm(Sk))
    FLambda0 = FLambda(xk, l0, Sk)
    FLambdaPlus = FLambda(xk, l0+deltaL, Sk)
    FLambdaMinus = FLambda(xk, l0-deltaL, Sk)
    
    coef = 0
    if FLambda0 < FLambdaPlus and FLambda0 < FLambdaMinus:
        raise Exception("Sven: both Lambdas are increasing funcion")
    if FLambda0 > FLambdaPlus and FLambda0 > FLambdaMinus:
        raise Exception("Sven: both Lambdas are deacreasing function")
    if FLambda0 > FLambdaPlus:
        coef = 1
        currentFValue = FLambdaPlus
        l1 = l0 + coef * deltaL
    if FLambda0 > FLambdaMinus:
        coef = -1
        currentFValue = FLambdaMinus
    
    l1 = l0 + coef * deltaL
    currentlValue = l1
    lambdaValues = [l0, l1]
    FNewLambda = 0
    n = 1
    while True:
        newLambda = currentlValue + 2*n*coef * deltaL
        lambdaValues.append(newLambda)
        FNewLambda = FLambda(xk, newLambda, Sk)
        # print(f"{newLambda}: {FNewLambda}")
        if (currentFValue < FNewLambda):
            break
        currentlValue = newLambda
        currentFValue = FNewLambda
        n = 2*n
    print(f"Sven Lambdas: {lambdaValues}")
    return np.array([lambdaValues[-3], (lambdaValues[-2] + lambdaValues[-1]) / 2])


sven(np.array([-22, -22]), np.array([0, 1]))
    

9.333809511662427: 4470.073335203986
21.778888860545663: 4102.331115475968
46.66904755831214: 7083.966676019931
[0, 3.111269837220809, 9.333809511662427, 21.778888860545663, 46.66904755831214]
[9.333809511662427, 34.223968209428904]


In [74]:
x0 = np.array([12, 10])

h = 100
def MOPlconst(e):
    xk = x0
    l = 1
    Fxk = f(xk)
    countGrad = True
    while True:
        if countGrad:
            gradFxk = fderright(xk, Fxk, h)# fdercenter(xk, h)
        xk_new = xk + l * (-gradFxk / norm(gradFxk))
        Fxk_new = f(xk_new)
        
        print(f'{xk_new}, {l}, {Fxk}, {Fxk_new}')
        
        if stopCriteria1(xk, Fxk, xk_new, Fxk_new, e):
            return xk_new
        if (Fxk <= Fxk_new):
            # What should I do with L?
            l = l / 2
            countGrad = False
        else: 
            xk = xk_new
            Fxk = Fxk_new
            countGrad = True

print(MOPlconst(0.001))
print(functionCalculated)

[11.49750798  9.13541815], 1, 307, 265.5912669491328
[10.99281217  8.27212091], 1, 265.5912669491328, 230.95072880497202
[10.48589142  7.41012824], 1, 230.95072880497202, 203.0697309623312
[9.97672424 6.5494606 ], 1, 203.0697309623312, 181.93947235993397
[9.46528876 5.69013892], 1, 181.93947235993397, 167.55100111585028
[8.95156272 4.83218463], 1, 167.55100111585028, 159.89520997807438
[8.43552346 3.97561971], 1, 159.89520997807438, 158.96283158011164
[7.91714791 3.12046663], 1, 158.96283158011164, 164.74443349076017
[8.17633568 3.54804317], 0.5, 158.96283158011164, 161.03163345223373
[8.30592957 3.76183144], 0.25, 158.96283158011164, 159.79173274537212
[8.37072652 3.86872557], 0.125, 158.96283158011164, 159.3259072200417
[8.40312499 3.92217264], 0.0625, 158.96283158011164, 159.13152566440164
[8.41932422 3.94889617], 0.03125, 158.96283158011164, 159.04396768833786
[8.42742384 3.96225794], 0.015625, 158.96283158011164, 159.00259690074503
[8.43147365 3.96893882], 0.0078125, 158.962831580